In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
p = 'output/2021_Feb_10_14_53_14_PAMBUH/'

In [ ]:
import os
import sys
os.chdir('..')
sys.path.append(p  + 'src')

In [ ]:
from pathlib import Path
from PIL import Image
from functools import partial

import cv2
import numpy as np
import torch
import torch.nn.functional as F
import albumentations as albu

import data
import augs
import callbacks
import sampler
import model as nn_model
from config import cfg, cfg_init

import  matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# datasets = data.build_datasets(cfg)
# dls = data.build_dataloaders(cfg, datasets, pin=True, drop_last=False)
# tdl = dls['TRAIN']
# xb,yb = next(iter(tdl))
# xb.shape, yb.shape

# s = sampler.tif_block_read('./input/hm/test/b2dc8411c.tiff')
# _,_,img = next(iter(s))
# img.shape, img.dtype

In [ ]:
def parse_model_path(p):
    name = str(p.name)
    epoch = name.split('_')[0]
    return int(epoch[1:])

def get_last_model(src):
    # assumes that model name is of type e500_blabla.pth, sorted by epoch #500
    models = list(Path(src).glob('*.pth'))
    res = []
    for i, m in enumerate(models):
        epoch = parse_model_path(m)
        #print(m, epoch)
        
        res.append([i,epoch])
    idx = sorted(res, key=lambda x: -x[1])[0][0]
    return models[idx]

def rescale(batch_img, scale): return F.interpolate(batch_img, scale_factor=(scale, scale))

def preprocess_image(cfg, img):
    train_trans = augs.get_aug('light', cfg.TRANSFORMERS)
    transform = train_trans.transforms.transforms[-1]
    #print(transform)
    #transform = albu.Compose([albu.Normalize(mean=cfg.TRANSFORMERS.MEAN, std=cfg.TRANSFORMERS.STD)])
    ch, H,W, dtype = *img.shape, img.dtype
    assert ch==3
    assert dtype==np.uint8
    img = img.transpose(1,2,0)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (W//2, H//2))
    return transform(image=img)['image']
    
def _infer_func(imgs, cfg, model):
    batch = []
    for img in imgs:
        batch.append(preprocess_image(cfg, img))
    
    batch = torch.stack(batch,axis=0)
    #print(batch.shape, batch.dtype)
    with torch.no_grad():
        res = torch.sigmoid(model(batch))
    res = rescale(res, 2)
    return res

def get_infer_func(p):
    cfg_init(p + 'cfg.yaml')
    cfg['PARALLEL']['DDP'] = False
    cfg['DATA']['TRAIN']['PRELOAD'] = False
    # TODO replace this
    cfg['TRANSFORMERS']['MEAN'] = (0.6616420882978906, 0.40941636273207577, 0.609469758520619)
    cfg['TRANSFORMERS']['STD'] = (0.10016240762553758, 0.17558692487633998, 0.12962072919146864)
    
    model_path = get_last_model(p + 'models')
    m = nn_model.load_model(cfg, str(model_path))
    
    return partial(_infer_func, cfg=cfg, model=m)

In [ ]:
foo = get_infer_func(p)

# Tests

## Sampler

In [ ]:
s = sampler.tif_block_read('./input/hm/test/b2dc8411c.tiff', block_size=(1024,1024))
s = iter(s)
_,_,img = next(s)
print(img.shape, img.dtype)

In [ ]:
Image.fromarray(img.transpose(1,2,0)).resize((256,256))

## Rasterio

In [ ]:
import rasterio as rio

In [ ]:
ds = rio.open('./input/hm/train/095bf7a1f.tiff')
ds.shape

In [ ]:
x,y = 15750,16100
w,h=1024,1024
img = ds.read([1,2,3], window=((x,x+w),(y,y+h)))
print(img.shape, img.dtype)

In [ ]:
Image.fromarray(img.transpose(1,2,0)).resize((256,256))

In [ ]:
bb = foo([img])
bb.shape, bb.dtype, bb.max()

In [ ]:
plt.imshow(bb[0][0])

In [ ]:
t = callbacks.denorm(dxb[0], mean=cfg.TRANSFORMERS.MEAN, std=cfg.TRANSFORMERS.STD).squeeze()
t = t.squeeze().permute(1,2,0).numpy()
t.shape

Image.fromarray((255.*t).astype(np.uint8))#.resize((512,512))